In [ ]:
# pip install python-bidi
# pip install hazm

In [38]:
from tqdm.notebook import tqdm
import numpy as np
import re
import string
from hazm import word_tokenize
from hazm import sent_tokenize
from hazm import Normalizer
from collections import Counter

### Learn about the text before starting to clean and preprocess it.

In [39]:
with open('./persian_combined_text.txt', 'r', encoding="utf-8") as f:
    dirty_corp = f.read()
dirty_corp = dirty_corp.split('^^^^^^')
print(f'Num of texts: {len(dirty_corp)}')

Num of texts: 1382


Please note that the actuall number of texts here is 2000. It is just because when converting pdf files to  word files we sometimes combined more than one pdf file in one word file

In [73]:
print(dirty_corp[1][1:1000])

صفحه: 1  3.۱۳ ۷۷۷۷/۷۷/۰۳۳۰ در بو تس اگاه دا ود یبای نا یکی وی  برای دریافت کتابهای بیشتر به آدرس بالا مراجعه کنید  تمامی حقوق برای تاریخ ما محفوظ است ات ۵ و  کتابخانه مجزی «تزیخ ما» نخستین پایگاه دانلود کتابهاي تزیخی و مذهبی می باشد که زمان احداث آن به سال 1386 بز می گردد و تاکنون بسیزی از کتب تزیخی و مذهبی را به صورت الكترونيکي (۳0۴) بر روي دنياي مجزي منتشر نموده است.  ۱۵:5 ۷/۵۵۵ ۰ ۵[۱.001 ۳۵2۵۲۱۰۴۱۵01 : از۵ ۴۳ 7۲ م۵ ۲۲۵://۸۵۱۵۱.۱۲ :۷۷۶۵۵ ۵۵۲ ۵۸0۲655 ۴6۵۵۴  ۷۷۷۷۷۷۰۳۳۰۱۵۲۱۱۱ ۱۲   صفحه: 2  تاریخ و اساطیر تطبیقی ایران باستان )۱(  سیمای واقعی زرتشت تاریخی  به همراه معرفی چهرةٌ تاریخی انبیای کتب مقدس (تلاشی در پایان نهادن بر عهد اساطیر)  در مورد زرتشت و اوستا ق تاریخ اساطیری ایران  ۲۱۰۰۵ شهر لینشوپینگ. سوند  نفس باد صبا مشک فشان خواهد شد عالم پیر دگر باره جوان خواهد شد   صفحه: 3  لب مطلب پیام عیسی مسیح تاریخی یعنی یهودای جلیلی فرزند زیپورایی این بود که به جز خدای یکتا نپرستید و از جهانخواران اطاعت نکرده و بدیشان باج ندهید.  عنوان کتاب و يا کتابهای آسمانی که افراط و تفریط در متون آنها تباه


In [41]:
#Count the number of words in the file
num_words = sum(len(doc.split()) for doc in dirty_corp)
#num_words
print("The file contains", num_words, "words.")

The file contains 110037959 words.


In [42]:
word_counts = Counter() 
for line in dirty_corp:
    words = word_tokenize(line.strip())   # tokenize each line into words
    word_counts.update(words)

most_common_words = word_counts.most_common(10)
print("The 10 most common words in the file are:")
for word, count in most_common_words:
    print(word, count)

The 10 most common words in the file are:
و 4908774
. 4586423
در 2387302
که 2309124
از 2260401
» 2034824
به 1948159
: 1833595
را 1601446
( 1213648


Because the text is still unclean, we get the above most common words, which is a mess))

So, we will clean the text and run this code later.

In [43]:
import string
import re

class TextCleaner():
    """
    Persian text cleaner
    """
    def __init__(self):
        self.min_ascii = '0600'
        self.max_ascii = '06FF'
        self.punctuation = string.punctuation + '،' + '؛' + '؟' + '؛' + '۔' + '»' + '«' + '-'
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        self.text = None
        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
    
    def remove_punct(self, text):
        # Remove patterns of the form "number: صفحه"
        text = re.sub(r'صفحه\s*:\s*\d+', '', text)
        # Remove all other punctuation
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        # Remove all numbers
        num_pattern = r'[\u06F0-\u06F9]'
        text = re.sub(num_pattern, ' ', text)
        return text
    
    def remove_spaces(self, text):
        # Remove extra spaces
        try:
            text = re.sub(self.one_space_regex, ' ', text)    
            text = text if text[0] != ' ' else text[1:]
            text = text if text[-1] != ' ' else text[:-1]
            return text
        
        except IndexError as e:
            return ''
    
    def is_fa_token(self, token):
        # Check if token is in Persian language
        for ch in set(token):
            if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
                return False
        return True

    def remove_foreign_lang(self, text):
        # Remove non-Persian tokens
        clean_text = ''
        for token in text.split():
            if self.is_fa_token(token):
                clean_text += ' ' + token
        return clean_text[1:]
    
    
    def clean_text(self, text):
        text = self.remove_num(text)
        text = self.remove_punct(text)
        text = self.remove_spaces(text)
        text = self.remove_foreign_lang(text)
        text = self.remove_spaces(text)
        return text

In [44]:
cleaned_texts = []

text_cleaner = TextCleaner()

for text in dirty_corp:
    cleaned_text = text_cleaner.clean_text(text)
    cleaned_texts.append(cleaned_text)


In [45]:
len(cleaned_texts)

1382

In [69]:
#print(cleaned_texts[1][1:100])

## Removing Stopwords

[Stopwords were taken from here]("https://github.com/mhbashari/awesome-persian-nlp-ir")

In [48]:
# Load the stopwords from the text file
with open("Persian-stopwords.txt", "r", encoding="utf-8") as f:
    stop_words = [line.strip() for line in f]

In [49]:
#stop_words

In [50]:
cleaned_corp= []

for text in cleaned_texts:
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    cleaned_corp.append(' '.join(filtered_words))

In [51]:
word_counts = Counter() 
for line in cleaned_corp:
    words = word_tokenize(line.strip())   # tokenize each line into words
    word_counts.update(words)

most_common_words = word_counts.most_common(10)
print("The 10 most common words in the file are:")
for word, count in most_common_words:
    print(word, count)

The 10 most common words in the file are:
سال 173800
ایران 168728
کار 142945
دست 137521
بن 132340
تاریخ 110698
کتاب 107934
قرار 94694
شاه 84279
نظر 82769


## Normalization

In [52]:
from hazm import *
normalizer = Normalizer()

normalized_texts = []
for text in cleaned_corp:
    normalized_text= normalizer.normalize(text)
    normalized_texts.append(normalized_text)

## Stemming & Lemmatization 

In [53]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

stemmed_lemmatized_texts = []

for text in normalized_texts:
    stemmed_text = [stemmer.stem(word) for word in text.split()]
    lemmatized_text = [lemmatizer.lemmatize(word) for word in stemmed_text]
    stemmed_lemmatized_text = ' '.join(lemmatized_text)
    stemmed_lemmatized_texts.append(stemmed_lemmatized_text)

### An overview how the text looks before and after cleaning

In [66]:
initial_text = dirty_corp[1][1:2000]
final_text = stemmed_lemmatized_texts[1][1:2000]
print(f'Before: {initial_text}')
print('\n')
print(f'After: {final_text}')

Before: صفحه: 1  3.۱۳ ۷۷۷۷/۷۷/۰۳۳۰ در بو تس اگاه دا ود یبای نا یکی وی  برای دریافت کتابهای بیشتر به آدرس بالا مراجعه کنید  تمامی حقوق برای تاریخ ما محفوظ است ات ۵ و  کتابخانه مجزی «تزیخ ما» نخستین پایگاه دانلود کتابهاي تزیخی و مذهبی می باشد که زمان احداث آن به سال 1386 بز می گردد و تاکنون بسیزی از کتب تزیخی و مذهبی را به صورت الكترونيکي (۳0۴) بر روي دنياي مجزي منتشر نموده است.  ۱۵:5 ۷/۵۵۵ ۰ ۵[۱.001 ۳۵2۵۲۱۰۴۱۵01 : از۵ ۴۳ 7۲ م۵ ۲۲۵://۸۵۱۵۱.۱۲ :۷۷۶۵۵ ۵۵۲ ۵۸0۲655 ۴6۵۵۴  ۷۷۷۷۷۷۰۳۳۰۱۵۲۱۱۱ ۱۲   صفحه: 2  تاریخ و اساطیر تطبیقی ایران باستان )۱(  سیمای واقعی زرتشت تاریخی  به همراه معرفی چهرةٌ تاریخی انبیای کتب مقدس (تلاشی در پایان نهادن بر عهد اساطیر)  در مورد زرتشت و اوستا ق تاریخ اساطیری ایران  ۲۱۰۰۵ شهر لینشوپینگ. سوند  نفس باد صبا مشک فشان خواهد شد عالم پیر دگر باره جوان خواهد شد   صفحه: 3  لب مطلب پیام عیسی مسیح تاریخی یعنی یهودای جلیلی فرزند زیپورایی این بود که به جز خدای یکتا نپرستید و از جهانخواران اطاعت نکرده و بدیشان باج ندهید.  عنوان کتاب و يا کتابهای آسمانی که افراط و تفریط در متون آن

In [56]:
with open('persian_cleaned_corpus.txt', 'w', encoding='utf-8') as f:
    for text in stemmed_lemmatized_texts:
        f.write(text + '\n')

## Stemming

Here is another method for stemming that can be applied instead of hazm stemming. 
[PersianStemmer can be fouund here]("https://github.com/htaghizadeh/PersianStemmer-Python")

In [36]:
# !pip install PersianStemmer
# !pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade

In [37]:
# from PersianStemmer import PersianStemmer 

# # Define the stemmer
# stemmer = PersianStemmer()

# # Stem each text in the filtered_texts list
# stemmed_texts = []
# for text in cleaned_corp:
#     stemmed_text = ' '.join(stemmer.stem(w) for w in text.split())
#     stemmed_texts.append(stemmed_text)

# # Assign the stemmed texts to the original variable name
# cleaned_corp = stemmed_texts
